In [1]:
import numpy as np
import tifffile
from skimage.filters import gaussian, threshold_otsu
import os
import glob
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define input and output paths
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/Static-A-1/Golgi'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Golgi_2'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Find all .tif files in the input directory
input_files = glob.glob(os.path.join(input_dir, '*.tif'))
print(f"Found {len(input_files)} .tif files to process")

# Process each file
for input_file in input_files:
    # Get base filename for output
    filename = os.path.basename(input_file)
    base_name = os.path.splitext(filename)[0]

    print(f"Processing: {filename}")

    # Load the image
    img = tifffile.imread(input_file)
    print(f"  Image shape: {img.shape}")

    # Extract Golgi channel based on image format
    if len(img.shape) == 2:
        print("  Detected single-channel image, using as is")
        golgi_channel = img
    elif len(img.shape) == 3 and img.shape[0] == 3:
        print("  Detected format: (C, H, W)")
        golgi_channel = img[2]
    elif len(img.shape) == 3 and img.shape[2] == 3:
        print("  Detected format: (H, W, C)")
        golgi_channel = img[:, :, 2]
    else:
        print(f"  Unexpected image shape: {img.shape}. Using first channel/plane.")
        golgi_channel = img[0] if img.shape[0] < img.shape[1] else img[:, :, 0]

    # Apply Gaussian blur for noise reduction
    print("  Applying Gaussian blur...")
    sigma = 1.0
    golgi_smoothed = gaussian(golgi_channel, sigma=sigma)
    print(f"  Gaussian blur applied with sigma={sigma}")

    # Apply Otsu thresholding
    print("  Performing Otsu thresholding...")
    thresh = threshold_otsu(golgi_smoothed)
    thresh_adj = thresh * 2.1
    binary = golgi_smoothed > thresh_adj

    # Save the processed binary image
    binary_output = os.path.join(output_dir, f"{base_name}_golgi_adjusted_binary.tif")
    tifffile.imwrite(binary_output, binary.astype(np.uint8) * 255)  # Convert to 8-bit image
    print(f"  Saved binary image to {binary_output}")

print("All processing complete!")


Mounted at /content/drive
Found 16 .tif files to process
Processing: denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Golgi_contrast_bg_tophat.tif
  Image shape: (1024, 1024)
  Detected single-channel image, using as is
  Applying Gaussian blur...
  Gaussian blur applied with sigma=1.0
  Performing Otsu thresholding...
  Saved binary image to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Golgi_2/denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Golgi_contrast_bg_tophat_golgi_adjusted_binary.tif
Processing: denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_Golgi_contrast_bg_tophat.tif
  Image shape: (1024, 1024)
  Detected single-channel image, using as is
  Applying Gaussian blur...
  Gaussian blur applied with sigma=1.0
  Performing Otsu thresholding...
  Saved binary image to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Golgi_2/denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_Golgi_contrast_bg_tophat_golgi_adjusted_binary.tif
P